In [1]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import pandas as pd
import seaborn as sns
#from scipy import stats

from func_utils import *

#Two lines to make project autoimport again after 
#every modifications in func_utils
%load_ext autoreload
%autoreload 2

# Lettura del documento

In [2]:
from func_utils import getDataFilePath
df = pd.read_excel(getDataFilePath(), sheet_name = 'Tabella')

# Filtro dati



In [3]:
valid_claim_values = ['ZW01','ZW02','ZW06']
valid_part_number_descr_values = ['-ECONOMY', '-ECONOMY50', '-E1', '-E3']

#filter by claim_type and part_number_des
filt_df = filterByColumnValues(df,'claim_type',valid_claim_values)
filt_df = filterByColumnValues(filt_df,'part_number_des',valid_part_number_descr_values)

filt_df

,id_claim_hash,claim_type,claim_status,claim_status_des,work_hours,failure_date,vin_hash,product_division,business_unit_des,region,...,component,component_des,item_type,id_causal_part_hash,id_part_number_hash,part_number_des,cd_decision,decision_des,amount,cd_dealer_hash
0,70a6c41c37a072a07164100b1182300149231993c9604d...,ZW02,Z080,Claim paid,0.20,2019-02-28,7844f9bafe2de98fbcb6c501885449218aceb8c0c315db...,TRUCK,COMMERCIAL VEHICLES B.U.,Europe,...,54063,ROCKER TOP COVER,FR,c47636bcc77194a7fbfa08d10d5f18be88e2654de170d0...,5cc870b50d68a02f026216d8d089eb8615e8e26129308d...,-ECONOMY,P,Pay,5.906040,35e183f47f75b85aa0c92e6251ef8c3d1bbf184708a529...
2,41311833ad1442619fad570eb3186e2532dcd22489eabf...,ZW02,Z080,Claim paid,0.60,2019-12-02,6527fb44ac31cdce99bc39fcec7d947e552eee4abe8e14...,TRUCK,BU TRUCKS -HEAVY RANGE,Europe,...,76081,STARTER MOTOR ASSY,FR,140356535f38645fbae99ad8816575da6e91e634089873...,5cc870b50d68a02f026216d8d089eb8615e8e26129308d...,-ECONOMY,P,Pay,25.080000,c18a8ad752fb7e649161dcabed2d1fb96fa38265866c1e...
3,178303f7069ee3d0650a50ac6491de917efad040cc10fb...,ZW02,Z080,Claim paid,2.00,2020-06-12,dc30d2d67f63c15ed647c96de9794aed4b37306f66cbc7...,TRUCK,BU TRUCKS -HEAVY RANGE,Europe,...,55292,MUDGUARD,FR,6d9f59542772b3c15939a30ff690d0dc1705180406528f...,5cc870b50d68a02f026216d8d089eb8615e8e26129308d...,-ECONOMY,P,Pay,114.600000,40f6461449ca9c11a4e7bf125b67626759976a171b4a72...
4,4c379212dd6466ae8d1cc9365a6facc05c21c1c6c8fde9...,ZW01,Z080,Claim paid,0.10,2021-07-20,19e83fb7cfd3a24b4ec4159a22d1bca823485f4f59f46a...,BUS,COMMERCIAL VEHICLES B.U.,Europe,...,50471,ELECTRIC DECELERATOR,FR,b4f0f24a67fd7bdc5284caac0d8409678b95eb6e2a5ba5...,5cc870b50d68a02f026216d8d089eb8615e8e26129308d...,-ECONOMY,P,Pay,3.100000,4eb70fde548ac04fb216d9ea04721497db4a4f7b2f6ea8...
5,ba855dc9287b23b47d929db58cf4be351d0620fe4d74f7...,ZW01,Z080,Claim paid,0.50,2020-08-03,57ec655a71291f290cca3f9e64f27b2d3188a5555c8dad...,TRUCK,COMMERCIAL VEHICLES B.U.,Europe,...,54241,TURBO CHARGER,FR,758dc31d7430b3c566ce5bdec59af7e2b8a6778024471f...,5cc870b50d68a02f026216d8d089eb8615e8e26129308d...,-ECONOMY,P,Pay,20.150000,5f3b6732c285356be83ff90d5f3cbc5a3afe11add07d57...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644542,4ab775abcfc07af3cff1eaec7e3268517e9fcec78a56b9...,ZW02,Z080,Claim paid,4.65,2019-02-11,6ca114f977053f48573a371ff439524865c953d7005ccf...,TRUCK,BU TRUCKS -MEDIUM RANGE,Europe,...,50743,NOX DETECTION SENSOR,FR,8a6ce6993469b0044dbd525a5b6656aa626cea4902b867...,5cc870b50d68a02f026216d8d089eb8615e8e26129308d...,-ECONOMY,P,Pay,259.010000,4b050802135759e35dc2bb285005e7a5ea8f4ec8f8b515...
644543,f0744e92e666002e85be195155dd0e9339d5dab119a416...,ZW02,Z080,Claim paid,1.20,2019-05-30,08dbfa6b2c6bb0031df7ed661fdb03b922524db363c3d0...,TRUCK,COMMERCIAL VEHICLES B.U.,Europe,...,50146,HYDR.ST.SYS PIPES+HOSES,FR,50938cfdce8595a5f29062abdcaffffc0151e9d7f42566...,5cc870b50d68a02f026216d8d089eb8615e8e26129308d...,-ECONOMY,P,Pay,84.558140,0ac8f96330b2fadef0f4f052dc36e91075b89998de00d4...
644544,5d6969a2ad9a348a582c8866970b84dbc3373bdeb0359c...,ZW02,Z080,Claim paid,0.10,2021-01-25,3085e86f43483773365dfa04250c4844142e4899d5e3fb...,TRUCK,COMMERCIAL VEHICLES B.U.,Europe,...,76201,BATTERY,FR,be489a7ed23065e5f5341c367f7923c5b5d41caa90ca82...,5cc870b50d68a02f026216d8d089eb8615e8e26129308d...,-ECONOMY,P,Pay,8.100000,de3d0af8236f47c31d41ff2239626659ece35033c428e0...
644545,0530495825d7a8fa90a3d1a8b933164024ce69943e3732...,ZW01,Z080,Claim paid,1.00,2021-01-20,c27a1de4ffaed8e4c09a28b83ebe6fb00ebf375d85d0c6...,BUS,BUS DIVISION,Europe,...,50604,"COLLARS,HOSES,SLEEVES",FR,9b2b4b307c2a19bd1ff2c0f1bb56ccbec3dbd383709ba0...,5cc870b50d68a02f026216d8d089eb8615e8e26129308d...,-ECONOMY,P,Pay,76.000000,cc4116d9cf1e2d54ba6644be17e845db0b26716bb99a45...


# Filtro per nazioni

In [9]:
countries_list=getTableColumn(filt_df,'country',True)#Seleziono tutte le possibili nazioni che compaiono
#lf=[None] * len(countries_list) #Creo una lista di lunghezza identica a quella di countries_list contente tutti i dati per ogni nazione
#for i in range(len(countries_list)):
#    lf[i]=filt_df1[filt_df1['country'].isin([countries_list[i]])]
df_list = separateDfByCountry(filt_df,countries_list)

#countries_list
#df_list[0]

array(['Poland', 'Italy', 'Spain', 'Croatia', 'Germany', 'France',
       'Czech Republic', 'Austria', 'Switzerland', 'Slovakia', 'Romania',
       'United Kingdom', 'Netherlands', 'Portugal', 'Norway', 'Finland',
       'Denmark', 'Belgium', 'Hungary', 'Bulgaria', 'Lithuania', 'Sweden',
       'Slovenia', 'Iceland', 'Serbia', 'Ireland', 'Ukraine', 'Latvia',
       'Estonia', 'Greece', 'Bosnia-Herz.', 'Moldavia', 'Montenegro',
       'Faroe Islands', 'Cyprus'], dtype=object)

# Analisi Dati

In [17]:
#Cazzo è kappa dai (eh mo non è che mi devo ricordare che nazione era la terza dell'array cioèèèè)
# Manco un commento per segnalare che nazione è ahahaha
k=2# Seleziona la nazione 

spainIndex = getValueIndex(countries_list,'Spain') 
amount_array = df_list[spainIndex]["amount"].to_numpy() #Estraggo rispettivamente gli array contente il guadagno ottenuto
workhours_array=df_list[spainIndex]["work_hours"].to_numpy() #Estraggo rispettivamente il numero di ore di lavoro

# ISTOGRAMMI

## Istogrammi che riportanno rispettivamente il numero di occorrenze delle variabili "Amount" e "Hours"

## Problema: Ridurre la dimensione della griglia

In [ ]:
x = amount_array
bins = np.linspace(0,10000, 100);
plt.hist(x, bins,label='Amount')  
plt.yscale('log')
plt.ylabel('Number')
plt.xlabel('Amount')
plt.show()

In [ ]:
y=workhours_array;
bins = np.linspace(0,100, 150);
plt.yscale('log');
plt.ylabel('Number');
plt.xlabel('Hours');
plt.grid()
plt.hist(y, bins,density=None, label='Hours')
plt.show()


# Fit Lineari

## Filtra con i most_common()

In [ ]:
elementi= 25 #Selezionare il numero di componenti da analizzare

In [ ]:
array2=[None] * elementi 
for i in range(elementi):
    array2[i]=Counter(filt_df1['component']).most_common(elementi)[i][0]


In [ ]:
new=[None] * len(countries_list) #Creo una lista di lunghezza identica a quella di "a" contente tutti i dati per ogni nazione
for i in range(len(countries_list)):
    new[i]=lf[i][lf[i]['component'].isin(array2)]

In [ ]:
new[2]['country']

## Filtro ulteriore escludendo i valori che hanno zero ore di lavoro e guadagno netto e viceversa

In [ ]:
new1_filt=new[(new1['work_hours']>0) & (new1['amount']>0)]

In [ ]:
outliers_1=new1[(new1['work_hours']==0) | (new1['amount']==0)]

In [ ]:
print('Il numero di dati scartati é:', len(outliers_1))

## Plot

In [ ]:
sns.relplot(
    data=new1_filt,
    x="work_hours", y="amount",
    hue="component", style="product_division"
)

## Fit per componente

In [ ]:
sns.lmplot(x="work_hours", y="amount", hue="component", data=new1_filt);

# Outliers

In [ ]:
new1_filt['amount/work_hours']= new1_filt['amount']/new1_filt['work_hours']

In [ ]:
print("Numero di dati a disposizione: ",len(new1_filt))

## Box-Plot per nazione

In [ ]:
sns.boxplot(x=new1_filt["amount/work_hours"],y=new1["component"])
unique_arr = new1_filt["country"].unique()
plt.title(unique_arr[0])
plt.show()